# Normalized Faces Scenario

This notebook creates a new scenario where the face of the person is the only thing visible in the video.

In [ ]:
import requests
import numpy as np

# Get the UV map from the internet
url = "https://raw.githubusercontent.com/apple2373/mediapipe-facemesh/main/data/uv_map.json"
response = requests.get(url)

uv_map_dict = response.json()
uv_map = np.array([(uv_map_dict["u"][str(i)], uv_map_dict["v"][str(i)]) for i in range(468)])

dim = 256
landmarks_uv = np.array([(dim * x, dim * y) for x, y in uv_map])

In [ ]:
import cv2
import mediapipe as mp

from mediapipe.tasks.python import vision
from skimage.transform import PiecewiseAffineTransform, warp
from mediapipe.tasks.python.core import base_options as base_options_module

base_options = base_options_module.BaseOptions(
    model_asset_path='../../data/mediapipe/face_landmarker_v2_with_blendshapes.task',
    delegate=base_options_module.BaseOptions.Delegate.CPU,
)
options = vision.FaceLandmarkerOptions(
    base_options=base_options,
    output_face_blendshapes=True,
    output_facial_transformation_matrixes=True,
    num_faces=1,
)


def normalize_face(frame, landmarks) -> np.ndarray:
    # https://scikit-image.org/docs/dev/auto_examples/transform/plot_piecewise_affine.html
    tform = PiecewiseAffineTransform()
    tform.estimate(landmarks_uv, landmarks)
    texture = warp(frame, tform, output_shape=(dim, dim))
    texture = (255 * texture).astype(np.uint8)

    return texture


def normalize_frames(frames: np.ndarray) -> np.ndarray:
    detector = vision.FaceLandmarker.create_from_options(options)

    height, width = frames[0].shape[0:2]

    processed_frames = []

    for idx, frame in tqdm(enumerate(frames), total=len(frames)):
        image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
        mesh_results = detector.detect(image)
        face_landmarks = mesh_results.face_landmarks[0]

        # after 468 is iris or something else
        face_landmarks = np.array([
            (width * point.x, height * point.y) for point in face_landmarks[:468]
        ])

        normalize_frame = normalize_face(frame, face_landmarks)
        processed_frames.append(normalize_frame)

    return np.array(processed_frames)

In [ ]:
def store_video_with_mask(frames: np.ndarray, filename: str, fps: int):
    shape = frames[0].shape

    fourcc = cv2.VideoWriter_fourcc(*'FFV1')
    out = cv2.VideoWriter(filename, fourcc, fps, (shape[1], shape[0]))

    for frame in frames:
        # Apply the mask to the frame
        frame = np.uint8(frame)

        # Convert the frame to BGR
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

        # Write the frame
        out.write(frame)

    out.release()

In [ ]:
import os
import subprocess
from respiration.dataset import VitalCamSet
from tqdm.auto import tqdm

dataset = VitalCamSet()
scenarios = dataset.get_scenarios(['101_natural_lighting'])

for (subject, setting) in scenarios:
    print(f"subject: {subject}")

    destination = os.path.join(dataset.data_path, subject, '303_normalized_face')

    # Copy the unisens data to the new scenario
    source = os.path.join(dataset.data_path, subject, setting, 'synced_Logitech HD Pro Webcam C920')
    os.makedirs(destination, exist_ok=True)
    subprocess.run(["cp", "-r", source, destination])

    frames, meta = dataset.get_video_rgb(subject, setting)
    frames = normalize_frames(frames)

    video_path = os.path.join(destination, 'Logitech HD Pro Webcam C920.avi')
    store_video_with_mask(frames, video_path, meta.fps)